# Temporary Notebook for developing new Data prep Process


In [1]:
from collections import defaultdict

import pandas as pd


In [2]:
# Local functions and classes
from types_and_classes import *
from utilities import *
from utilities import points_to_polygon
#from debug_tools import *
from debug_tools import make_sphere
#from structure_slice import *
#from structure_set import *
#from relations import *
from shapely.geometry import Polygon
from types_and_classes import Contour

### Global Settings

In [3]:
PRECISION = 2

In [4]:
%matplotlib inline

## Relationship Tests by Relationship Type

### Color Coding
<table style="border: 2px solid black; width=50px;"><tr><td>
<ul style="font-weight: 900; font-size: 20px;">
<li style="color: blue;">region a</li>
<li style="color: green;">region b</li>
<li style="color: orange;">intersection of a & b</li></ul>
</tr></td></>

## Contains

<link rel="stylesheet" href="relations.css">
<table width="350px">
<tr class="l"><th>Contains</th><th>Transitive</th></tr>
<td class="d" colspan="2">
All points of <span class="b">b</span> lie in the interior of <span class="a">a</span>, no points of <span class="b">b</span> lie in the exterior of <span class="a">a</span>, some points in <span class="a">a</span> are exterior to <span class="b">b</span>, and the boundaries of <span class="a">a</span> and <span class="b">b</span> do not intersect.
</td></tr>
<tr><td colspan="2">
<img src="Images/Relationships/contains simple.png" alt="Contains">
<img src="Images/Relationships/contains with hole.png" alt="Contains With Hole">
</td></tr>
</table></td></tr></table>

### Contains

#### Embedded Spheres
![Embedded Spheres](<Images/FreeCAD Images/Embedded Spheres.png>)

In [5]:
slice_spacing = 0.5
# Body structure defines slices in use
sphere6 = make_sphere(roi_num=1, radius=3, spacing=slice_spacing)
sphere3 = make_sphere(roi_num=2, radius=1.5, spacing=slice_spacing)
# combine the contours
slice_data = sphere6 + sphere3


In [6]:
contours = pd.DataFrame(slice_data)

contours['Polygon'] = contours['Points'].apply(points_to_polygon)
contours['Area'] = contours['Polygon'].apply(lambda poly: round(poly.area, PRECISION))


# Sort the DataFrame
contours.sort_values(by=['ROI', 'Slice', 'Area'], ascending=[True, True, False], inplace=True)

#contours.set_index(['ROI', 'Slice'], inplace=True, append=True)

In [7]:
contour_set = contours[contours.ROI == 1]
contour_slice = defaultdict(list)
contour_set.set_index('Slice', inplace=True)
for slice, contour in contour_set.Polygon.items():
    contours_on_slice = contour_slice[slice]
    new_contour = Contour(slice, contour, contours_on_slice)
    contour_slice[slice].append(new_contour)

In [8]:
contour_slice

defaultdict(list,
            {-3.0: [<types_and_classes.Contour at 0x27382cced10>],
             -2.5: [<types_and_classes.Contour at 0x27382ccee90>],
             -2.0: [<types_and_classes.Contour at 0x27382ccef10>],
             -1.5: [<types_and_classes.Contour at 0x27382ccf310>],
             -1.0: [<types_and_classes.Contour at 0x27382ccc950>],
             -0.5: [<types_and_classes.Contour at 0x27382ccf3d0>],
             0.0: [<types_and_classes.Contour at 0x27382ccf450>],
             0.5: [<types_and_classes.Contour at 0x27382ccf4d0>],
             1.0: [<types_and_classes.Contour at 0x27382ccf5d0>],
             1.5: [<types_and_classes.Contour at 0x27382ccf650>],
             2.0: [<types_and_classes.Contour at 0x27382ccf610>],
             2.5: [<types_and_classes.Contour at 0x27382ccf6d0>],
             3.0: [<types_and_classes.Contour at 0x27382ccf690>]})